# 🔬 Double Machine Learning 深度剖析

---

## 🎬 真实场景：当经济学家遇上机器学习

想象你是某电商平台的数据科学家，老板问你：

> 「我们的 VIP 会员权益到底值多少钱？能不能用因果推断算出来？」

你信心满满地说：「没问题！我有 500 个用户特征，用 XGBoost 估计倾向得分，再用 DR 估计效应！」

结果呢？

- 🔴 **效应估计忽高忽低**，置信区间巨大
- 🔴 **换个模型结果就变**，老板不敢信
- 🔴 **p-value 一直很显著**，但你心里没底

问题出在哪？

---

### 高维数据的诅咒 👻

当特征很多（高维）时，传统方法会遇到两个致命问题：

| 问题 | 传统方法的表现 | 后果 |
|------|---------------|------|
| **正则化偏差** | ML 模型需要正则化防止过拟合 | 正则化会把因果效应也「缩小」了 |
| **过拟合偏差** | 用同一份数据训练和预测 | 残差被低估，标准误失效 |

**Double Machine Learning (DML)** 就是为了解决这两个问题而生的！

---

## 📚 本节学习目标

完成本练习后，你将能够：

1. 🎯 理解**正则化偏差**和**过拟合偏差**的来源
2. 🎯 掌握 **Neyman 正交性**的核心思想
3. 🎯 **从零实现** Cross-fitting DML（不用任何因果库）
4. 🎯 理解 DML 的**渐近正态性**和置信区间构造
5. 🎯 在面试中清晰地解释 DML 原理

> ⚠️ **重要**: 这是大厂面试的高频考点！很多面试官会问「DML 和普通 DR 有什么区别？为什么需要 Cross-fitting？」

In [ ]:
# 导入必要的库
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LogisticRegression, Ridge, Lasso, LassoCV
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, RandomForestClassifier
from sklearn.model_selection import KFold, cross_val_predict
from sklearn.preprocessing import StandardScaler
from typing import Tuple, Dict, Callable, List
from scipy import stats
import warnings
warnings.filterwarnings('ignore')

# 设置
plt.rcParams['font.sans-serif'] = ['Arial Unicode MS', 'SimHei']
plt.rcParams['axes.unicode_minus'] = False
sns.set_style('whitegrid')
np.random.seed(42)

print("✅ 环境准备完毕！")
print("📦 本节我们将从零实现 Double ML，不依赖 econml 等库")

---

## 🧠 Part 1: 为什么需要 Double ML？

### 1.1 正则化偏差问题

假设真实的数据生成过程 (DGP) 是：

$$Y = \tau \cdot T + g(X) + \epsilon$$

其中：
- $\tau$ 是我们要估计的**因果效应**
- $g(X)$ 是协变量对结果的影响（可能非常复杂）
- $T$ 是处理变量

**问题来了**：如果我们直接用带正则化的回归：

$$\min_{\tau, \beta} \sum_i (Y_i - \tau T_i - X_i'\beta)^2 + \lambda ||\beta||$$

正则化项 $\lambda ||\beta||$ 会「顺便」把 $\tau$ 也往 0 压缩！

这就是**正则化偏差 (Regularization Bias)**。

In [ ]:
def demonstrate_regularization_bias():
    """
    演示正则化偏差：当特征很多时，Lasso 会把因果效应也压缩
    """
    np.random.seed(42)
    n = 1000
    p = 200  # 高维：200 个特征
    
    # 生成高维特征
    X = np.random.randn(n, p)
    
    # 真实的稀疏系数（只有前 10 个有用）
    true_beta = np.zeros(p)
    true_beta[:10] = np.random.randn(10)
    
    # 处理变量（与 X 相关）
    propensity_logit = X[:, 0] + 0.5 * X[:, 1]
    T = (np.random.rand(n) < 1/(1 + np.exp(-propensity_logit))).astype(float)
    
    # 真实因果效应
    TRUE_ATE = 2.0
    
    # 生成结果
    Y = TRUE_ATE * T + X @ true_beta + np.random.randn(n) * 0.5
    
    # 方法 1: OLS（无正则化，但在高维下不稳定）
    X_with_T = np.column_stack([T, X])
    try:
        ols_coef = np.linalg.lstsq(X_with_T, Y, rcond=None)[0]
        ols_ate = ols_coef[0]
    except:
        ols_ate = np.nan
    
    # 方法 2: Lasso（有正则化偏差）
    lasso = LassoCV(cv=5, max_iter=10000)
    lasso.fit(X_with_T, Y)
    lasso_ate = lasso.coef_[0]
    
    # 方法 3: 朴素估计
    naive_ate = Y[T==1].mean() - Y[T==0].mean()
    
    print("🔬 正则化偏差演示")
    print("=" * 50)
    print(f"设置: n={n}, p={p} (高维数据)")
    print(f"真实 ATE = {TRUE_ATE}")
    print()
    print(f"方法              | ATE 估计  | 偏差")
    print("-" * 50)
    print(f"朴素估计          | {naive_ate:8.4f} | {naive_ate - TRUE_ATE:+.4f}")
    if not np.isnan(ols_ate):
        print(f"OLS (无正则化)    | {ols_ate:8.4f} | {ols_ate - TRUE_ATE:+.4f}")
    print(f"Lasso (正则化)    | {lasso_ate:8.4f} | {lasso_ate - TRUE_ATE:+.4f} ⚠️")
    print()
    print("💡 观察: Lasso 的估计被压缩向 0，这就是正则化偏差！")
    
    return TRUE_ATE, naive_ate, lasso_ate

demonstrate_regularization_bias()

### 1.2 过拟合偏差问题

即使我们用 AIPW，如果用**同一份数据**训练模型和计算效应，还会有**过拟合偏差**：

$$\text{AIPW} = \frac{1}{n}\sum_i \left[ \hat{\mu}_1(X_i) - \hat{\mu}_0(X_i) + \frac{T_i(Y_i - \hat{\mu}_1(X_i))}{\hat{e}(X_i)} - \frac{(1-T_i)(Y_i - \hat{\mu}_0(X_i))}{1-\hat{e}(X_i)} \right]$$

问题在于：$\hat{\mu}(X_i)$ 是用包含第 $i$ 个样本的数据训练的，所以 $Y_i - \hat{\mu}(X_i)$ 会被**低估**！

这个「自己预测自己」的问题，会导致：
- 标准误被低估
- 置信区间过窄
- p-value 过于显著

---

## 📐 Part 2: DML 的核心思想

### 2.1 Partially Linear Model (PLR)

DML 最经典的应用场景是**部分线性模型**：

$$Y = \tau \cdot T + g(X) + \epsilon, \quad E[\epsilon|X, T] = 0$$
$$T = m(X) + \eta, \quad E[\eta|X] = 0$$

其中：
- $\tau$ 是低维参数（我们关心的因果效应）
- $g(X)$, $m(X)$ 是高维 nuisance 参数（烦人但必须处理的）

### 2.2 Neyman 正交性（核心！）

DML 的核心创新是构造一个**对 nuisance 参数不敏感**的矩估计。

**关键洞察**：先把 $g(X)$ 和 $m(X)$ 从 $Y$ 和 $T$ 中「去掉」，得到残差：

$$\tilde{Y} = Y - g(X) = \tau \cdot T + \epsilon$$
$$\tilde{T} = T - m(X) = \eta$$

然后用残差做回归：

$$\tilde{Y} = \tau \cdot \tilde{T} + \epsilon$$

$$\hat{\tau} = \frac{\sum_i \tilde{Y}_i \tilde{T}_i}{\sum_i \tilde{T}_i^2} = \frac{E[\tilde{Y} \cdot \tilde{T}]}{E[\tilde{T}^2]}$$

### 📐 核心公式

$$\hat{\tau}_{DML} = \frac{\frac{1}{n}\sum_{i=1}^{n}(Y_i - \hat{g}(X_i))(T_i - \hat{m}(X_i))}{\frac{1}{n}\sum_{i=1}^{n}(T_i - \hat{m}(X_i))^2}$$

| 符号 | 含义 |
|------|------|
| $\hat{g}(X)$ | 结果模型：$E[Y|X]$ 的估计 |
| $\hat{m}(X)$ | 倾向得分：$E[T|X]$ 的估计 |
| $Y - \hat{g}(X)$ | 结果残差 |
| $T - \hat{m}(X)$ | 处理残差 |

**直观理解**：就像做一个「双重残差回归」——先用 ML 模型把 $X$ 的影响去掉，再估计 $T$ 对 $Y$ 的效应。

### 2.3 为什么叫「Double」？

「Double」有两层含义：

1. **Double Residualization**：对 $Y$ 和 $T$ 都做残差化
2. **Double Robustness**：如果 $\hat{g}$ 或 $\hat{m}$ 之一正确，估计就是一致的

这与 AIPW 的双重稳健性是相通的！

---

## 📊 Neyman 正交性的数学推导

**Neyman 正交条件**: 矩函数对 nuisance 参数的导数为 0

DML 使用的矩函数：

$$\psi(W; \tau, \eta) = (Y - \tau \cdot T - g(X))(T - m(X))$$

其中 $\eta = (g, m)$ 是 nuisance 参数。

**正交性条件**:

$$\frac{\partial}{\partial \eta} E[\psi(W; \tau_0, \eta)] \Big|_{\eta=\eta_0} = 0$$

**证明**:

$$E[\psi(W; \tau, \eta)] = E[(Y - \tau T - g(X))(T - m(X))]$$

对 $g$ 求偏导：

$$\frac{\partial}{\partial g} E[(Y - \tau T - g(X))(T - m(X))] = -E[T - m(X)] = 0$$

（因为 $E[T|X] = m(X)$，所以 $E[T - m(X)] = 0$）

对 $m$ 求偏导：

$$\frac{\partial}{\partial m} E[(Y - \tau T - g(X))(T - m(X))] = -E[Y - \tau T - g(X)] = 0$$

（因为当 $\tau = \tau_0$, $g = g_0$ 时，$E[Y - \tau_0 T - g_0(X)] = 0$）

**关键洞察**：矩函数在真实参数处对 nuisance 参数不敏感！

这意味着：
- 即使 $\hat{g}$, $\hat{m}$ 有小误差
- 对 $\hat{\tau}$ 的影响是二阶的（$O(||\hat{\eta} - \eta||^2)$）
- 而不是一阶的！

这就是为什么 DML 在高维数据下仍能得到 $\sqrt{n}$-一致的估计和有效推断！

---

---

## 🔧 Part 3: Cross-fitting（交叉拟合）

### 3.1 为什么需要 Cross-fitting？

问题：如果用全部数据训练 $\hat{g}$ 和 $\hat{m}$，然后用同一批数据计算残差：

- 残差 $Y_i - \hat{g}(X_i)$ 会偏小（因为模型「见过」这个样本）
- 导致 $\hat{\tau}$ 有偏

### 3.2 Cross-fitting 的解决方案

```
数据分成 K 折：[Fold 1] [Fold 2] [Fold 3] ... [Fold K]

对于 Fold k:
  - 用其他 K-1 折训练模型 g_k, m_k
  - 对 Fold k 的样本计算残差

汇总所有折的残差，计算最终估计
```

这样，每个样本的残差都是用「未见过它的模型」计算的！

### 3.3 Cross-fitting 伪代码

```python
def cross_fit_dml(X, T, Y, n_folds=5):
    Y_residuals = np.zeros(n)
    T_residuals = np.zeros(n)
    
    for train_idx, test_idx in KFold(n_folds).split(X):
        # 在训练集上训练模型
        g_model.fit(X[train_idx], Y[train_idx])
        m_model.fit(X[train_idx], T[train_idx])
        
        # 在测试集上计算残差
        Y_residuals[test_idx] = Y[test_idx] - g_model.predict(X[test_idx])
        T_residuals[test_idx] = T[test_idx] - m_model.predict_proba(X[test_idx])[:, 1]
    
    # 用残差做回归
    tau_hat = (Y_residuals * T_residuals).sum() / (T_residuals ** 2).sum()
    return tau_hat
```

---

## 💻 Part 4: 从零实现 DML

现在让我们**从零实现**一个完整的 DML 估计器！

In [ ]:
def generate_high_dimensional_data(
    n: int = 2000,
    p: int = 100,
    true_ate: float = 2.0,
    seed: int = 42
) -> Tuple[np.ndarray, np.ndarray, np.ndarray, float]:
    """
    生成高维数据用于测试 DML
    
    DGP:
        T = sigmoid(X @ beta_t) + noise
        Y = true_ate * T + X @ beta_y + noise
    
    场景：VIP 会员权益评估
    - X: 用户特征（购买历史、浏览行为、人口统计等）
    - T: 是否成为 VIP
    - Y: 后续消费金额
    """
    np.random.seed(seed)
    
    # 生成协变量
    X = np.random.randn(n, p)
    
    # 稀疏系数（只有前 10 个特征有用）
    beta_t = np.zeros(p)
    beta_t[:10] = np.array([0.8, 0.6, 0.5, 0.4, 0.3, -0.3, -0.4, 0.2, 0.1, -0.2])
    
    beta_y = np.zeros(p)
    beta_y[:10] = np.array([1.5, 1.2, 0.8, 0.6, 0.4, -0.5, -0.3, 0.2, 0.3, -0.1])
    
    # 处理变量（二元）
    propensity_logit = X @ beta_t
    propensity = 1 / (1 + np.exp(-propensity_logit))
    T = np.random.binomial(1, propensity)
    
    # 结果变量
    Y = true_ate * T + X @ beta_y + np.random.randn(n) * 0.5
    
    print(f"📊 高维数据生成完成")
    print(f"   样本量: n = {n}")
    print(f"   特征数: p = {p}")
    print(f"   真实 ATE: {true_ate}")
    print(f"   处理组比例: {T.mean():.1%}")
    
    return X, T, Y, true_ate

In [ ]:
# 生成数据
X, T, Y, TRUE_ATE = generate_high_dimensional_data(n=2000, p=100)

In [ ]:
# TODO 1: 实现 Cross-fitting DML
# 这是本节最重要的练习！

def double_ml_plr(
    X: np.ndarray,
    T: np.ndarray,
    Y: np.ndarray,
    n_folds: int = 5,
    outcome_model = None,
    propensity_model = None
) -> Dict:
    """
    实现 Double Machine Learning for Partially Linear Model
    
    核心步骤：
    1. Cross-fitting: 将数据分成 K 折
    2. 对每一折，用其他折训练 g(X) 和 m(X)
    3. 计算残差 Y_res = Y - g(X), T_res = T - m(X)
    4. 用残差做回归估计 tau
    5. 计算标准误
    
    参数:
        X: 协变量矩阵 (n, p)
        T: 处理变量 (n,)
        Y: 结果变量 (n,)
        n_folds: 交叉拟合的折数
        outcome_model: 结果模型（默认 Ridge）
        propensity_model: 倾向得分模型（默认 LogisticRegression）
    
    返回:
        包含 tau, se, ci_lower, ci_upper, pvalue 的字典
    """
    n = len(Y)
    
    # 默认模型
    if outcome_model is None:
        outcome_model = Ridge(alpha=1.0)
    if propensity_model is None:
        propensity_model = LogisticRegression(max_iter=1000, C=0.1)
    
    # 存储残差
    Y_residuals = np.zeros(n)
    T_residuals = np.zeros(n)
    
    # TODO: 实现 Cross-fitting
    # 提示：使用 KFold 将数据分成 n_folds 折
    kf = KFold(n_splits=n_folds, shuffle=True, random_state=42)
    
    for train_idx, test_idx in kf.split(X):
        # === 你的代码开始 ===
        
        # Step 1: 分割数据
        X_train, X_test = None, None  # 👈 你的代码
        T_train, T_test = None, None  # 👈 你的代码
        Y_train, Y_test = None, None  # 👈 你的代码
        
        # Step 2: 训练结果模型 g(X) 并预测
        # 注意：这里预测的是 E[Y|X]，不是 E[Y|X,T]
        # 所以用全部训练数据（不区分处理组）
        g_model = Ridge(alpha=1.0)
        # 👈 你的代码: g_model.fit(X_train, Y_train)
        g_pred = None  # 👈 你的代码: g_model.predict(X_test)
        
        # Step 3: 训练倾向得分模型 m(X) 并预测
        m_model = LogisticRegression(max_iter=1000, C=0.1)
        # 👈 你的代码: m_model.fit(X_train, T_train)
        m_pred = None  # 👈 你的代码: m_model.predict_proba(X_test)[:, 1]
        
        # Step 4: 计算残差
        Y_residuals[test_idx] = None  # 👈 你的代码: Y_test - g_pred
        T_residuals[test_idx] = None  # 👈 你的代码: T_test - m_pred
        
        # === 你的代码结束 ===
    
    # 检查是否完成了 TODO
    if Y_residuals.sum() == 0 or T_residuals.sum() == 0:
        print("❌ 请完成上面的 TODO！")
        return None
    
    # Step 5: 用残差做回归估计 tau
    # tau = Cov(Y_res, T_res) / Var(T_res)
    #     = sum(Y_res * T_res) / sum(T_res^2)
    tau_hat = (Y_residuals * T_residuals).sum() / (T_residuals ** 2).sum()
    
    # Step 6: 计算标准误
    # 基于 influence function 的标准误
    psi = (Y_residuals - tau_hat * T_residuals) * T_residuals
    J = (T_residuals ** 2).mean()
    var_tau = (psi ** 2).mean() / (n * J ** 2)
    se = np.sqrt(var_tau)
    
    # 置信区间和 p-value
    ci_lower = tau_hat - 1.96 * se
    ci_upper = tau_hat + 1.96 * se
    z_stat = tau_hat / se
    pvalue = 2 * (1 - stats.norm.cdf(abs(z_stat)))
    
    return {
        'tau': tau_hat,
        'se': se,
        'ci_lower': ci_lower,
        'ci_upper': ci_upper,
        'pvalue': pvalue,
        'Y_residuals': Y_residuals,
        'T_residuals': T_residuals
    }

<details>
<summary>💡 点击查看参考答案</summary>

```python
# Step 1: 分割数据
X_train, X_test = X[train_idx], X[test_idx]
T_train, T_test = T[train_idx], T[test_idx]
Y_train, Y_test = Y[train_idx], Y[test_idx]

# Step 2: 训练结果模型 g(X) 并预测
g_model.fit(X_train, Y_train)
g_pred = g_model.predict(X_test)

# Step 3: 训练倾向得分模型 m(X) 并预测
m_model.fit(X_train, T_train)
m_pred = m_model.predict_proba(X_test)[:, 1]

# Step 4: 计算残差
Y_residuals[test_idx] = Y_test - g_pred
T_residuals[test_idx] = T_test - m_pred
```

</details>

In [ ]:
# 完整实现（用于后续测试）
def double_ml_plr_complete(
    X: np.ndarray,
    T: np.ndarray,
    Y: np.ndarray,
    n_folds: int = 5,
) -> Dict:
    """
    完整的 DML 实现
    """
    n = len(Y)
    Y_residuals = np.zeros(n)
    T_residuals = np.zeros(n)
    
    kf = KFold(n_splits=n_folds, shuffle=True, random_state=42)
    
    for train_idx, test_idx in kf.split(X):
        X_train, X_test = X[train_idx], X[test_idx]
        T_train, T_test = T[train_idx], T[test_idx]
        Y_train, Y_test = Y[train_idx], Y[test_idx]
        
        # 训练结果模型
        g_model = Ridge(alpha=1.0)
        g_model.fit(X_train, Y_train)
        g_pred = g_model.predict(X_test)
        
        # 训练倾向得分模型
        m_model = LogisticRegression(max_iter=1000, C=0.1)
        m_model.fit(X_train, T_train)
        m_pred = m_model.predict_proba(X_test)[:, 1]
        
        # 计算残差
        Y_residuals[test_idx] = Y_test - g_pred
        T_residuals[test_idx] = T_test - m_pred
    
    # 估计 tau
    tau_hat = (Y_residuals * T_residuals).sum() / (T_residuals ** 2).sum()
    
    # 标准误
    psi = (Y_residuals - tau_hat * T_residuals) * T_residuals
    J = (T_residuals ** 2).mean()
    var_tau = (psi ** 2).mean() / (n * J ** 2)
    se = np.sqrt(var_tau)
    
    ci_lower = tau_hat - 1.96 * se
    ci_upper = tau_hat + 1.96 * se
    z_stat = tau_hat / se
    pvalue = 2 * (1 - stats.norm.cdf(abs(z_stat)))
    
    return {
        'tau': tau_hat,
        'se': se,
        'ci_lower': ci_lower,
        'ci_upper': ci_upper,
        'pvalue': pvalue,
        'Y_residuals': Y_residuals,
        'T_residuals': T_residuals
    }

In [ ]:
# 运行 DML
dml_result = double_ml_plr_complete(X, T, Y, n_folds=5)

print("🔬 Double ML 估计结果")
print("=" * 50)
print(f"真实 ATE: {TRUE_ATE}")
print(f"DML ATE:  {dml_result['tau']:.4f} ± {dml_result['se']:.4f}")
print(f"95% CI:   [{dml_result['ci_lower']:.4f}, {dml_result['ci_upper']:.4f}]")
print(f"P-value:  {dml_result['pvalue']:.4e}")
print(f"偏差:     {dml_result['tau'] - TRUE_ATE:+.4f}")
print()
print(f"✅ 真实值 {TRUE_ATE} {'在' if dml_result['ci_lower'] <= TRUE_ATE <= dml_result['ci_upper'] else '不在'} 置信区间内")

---

## 📊 Part 5: 残差可视化

让我们可视化 DML 的核心——残差化过程。

In [ ]:
# 可视化残差
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

# 1. Y 残差分布
axes[0].hist(dml_result['Y_residuals'], bins=50, alpha=0.7, color='steelblue', edgecolor='black')
axes[0].axvline(0, color='red', linestyle='--', linewidth=2)
axes[0].set_xlabel('Y 残差 (Y - g(X))', fontsize=12)
axes[0].set_ylabel('频数', fontsize=12)
axes[0].set_title('结果残差分布', fontsize=14)

# 2. T 残差分布
axes[1].hist(dml_result['T_residuals'], bins=50, alpha=0.7, color='coral', edgecolor='black')
axes[1].axvline(0, color='red', linestyle='--', linewidth=2)
axes[1].set_xlabel('T 残差 (T - m(X))', fontsize=12)
axes[1].set_ylabel('频数', fontsize=12)
axes[1].set_title('处理残差分布', fontsize=14)

# 3. 残差散点图 + 回归线
axes[2].scatter(dml_result['T_residuals'], dml_result['Y_residuals'], 
               alpha=0.3, s=10, c='gray')

# 拟合线
T_res = dml_result['T_residuals']
Y_res = dml_result['Y_residuals']
x_line = np.linspace(T_res.min(), T_res.max(), 100)
y_line = dml_result['tau'] * x_line
axes[2].plot(x_line, y_line, 'r-', linewidth=2, label=f'τ = {dml_result["tau"]:.3f}')
axes[2].plot(x_line, TRUE_ATE * x_line, 'g--', linewidth=2, label=f'真实 τ = {TRUE_ATE}')

axes[2].set_xlabel('T 残差', fontsize=12)
axes[2].set_ylabel('Y 残差', fontsize=12)
axes[2].set_title('残差回归：DML 的核心', fontsize=14)
axes[2].legend()

plt.tight_layout()
plt.show()

print("💡 解读:")
print("   - 左图: Y 残差应该均值接近 0")
print("   - 中图: T 残差也应该均值接近 0")
print("   - 右图: 斜率就是 DML 估计的因果效应 τ")

---

## 🔄 Part 6: 对比有无 Cross-fitting

让我们验证 Cross-fitting 的重要性。

In [ ]:
def dml_without_crossfitting(X, T, Y):
    """
    没有 Cross-fitting 的 DML（错误做法！）
    用同一份数据训练和预测
    """
    n = len(Y)
    
    # 用全部数据训练
    g_model = Ridge(alpha=1.0)
    g_model.fit(X, Y)
    g_pred = g_model.predict(X)  # 用同一份数据预测！
    
    m_model = LogisticRegression(max_iter=1000, C=0.1)
    m_model.fit(X, T)
    m_pred = m_model.predict_proba(X)[:, 1]  # 用同一份数据预测！
    
    # 计算残差
    Y_residuals = Y - g_pred
    T_residuals = T - m_pred
    
    # 估计 tau
    tau_hat = (Y_residuals * T_residuals).sum() / (T_residuals ** 2).sum()
    
    # 标准误（会被低估！）
    psi = (Y_residuals - tau_hat * T_residuals) * T_residuals
    J = (T_residuals ** 2).mean()
    var_tau = (psi ** 2).mean() / (n * J ** 2)
    se = np.sqrt(var_tau)
    
    return {'tau': tau_hat, 'se': se}

In [ ]:
# 多次模拟对比
n_simulations = 50
results_with_cf = []
results_without_cf = []

print("运行模拟实验...")
for seed in range(n_simulations):
    X_sim, T_sim, Y_sim, _ = generate_high_dimensional_data(n=1000, p=50, seed=seed)
    
    # 有 Cross-fitting
    result_cf = double_ml_plr_complete(X_sim, T_sim, Y_sim, n_folds=5)
    results_with_cf.append(result_cf['tau'])
    
    # 无 Cross-fitting
    result_no_cf = dml_without_crossfitting(X_sim, T_sim, Y_sim)
    results_without_cf.append(result_no_cf['tau'])

print("模拟完成!")

In [ ]:
# 可视化对比
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# 直方图对比
axes[0].hist(results_with_cf, bins=20, alpha=0.7, label='有 Cross-fitting', color='green', edgecolor='black')
axes[0].hist(results_without_cf, bins=20, alpha=0.7, label='无 Cross-fitting', color='red', edgecolor='black')
axes[0].axvline(TRUE_ATE, color='black', linestyle='--', linewidth=2, label=f'真实 ATE = {TRUE_ATE}')
axes[0].set_xlabel('ATE 估计', fontsize=12)
axes[0].set_ylabel('频数', fontsize=12)
axes[0].set_title('Cross-fitting 的影响', fontsize=14)
axes[0].legend()

# 箱线图对比
data = [results_with_cf, results_without_cf]
bp = axes[1].boxplot(data, labels=['有 CF', '无 CF'], patch_artist=True)
bp['boxes'][0].set_facecolor('lightgreen')
bp['boxes'][1].set_facecolor('lightcoral')
axes[1].axhline(TRUE_ATE, color='black', linestyle='--', linewidth=2, label=f'真实 ATE = {TRUE_ATE}')
axes[1].set_ylabel('ATE 估计', fontsize=12)
axes[1].set_title('估计分布对比', fontsize=14)
axes[1].legend()

plt.tight_layout()
plt.show()

# 统计对比
print("\n📊 统计对比")
print("=" * 50)
print(f"真实 ATE: {TRUE_ATE}")
print(f"\n有 Cross-fitting:")
print(f"   均值: {np.mean(results_with_cf):.4f}")
print(f"   偏差: {np.mean(results_with_cf) - TRUE_ATE:+.4f}")
print(f"   标准差: {np.std(results_with_cf):.4f}")
print(f"\n无 Cross-fitting:")
print(f"   均值: {np.mean(results_without_cf):.4f}")
print(f"   偏差: {np.mean(results_without_cf) - TRUE_ATE:+.4f}")
print(f"   标准差: {np.std(results_without_cf):.4f}")

print("\n💡 结论: Cross-fitting 能显著减少偏差！")

---

## 🏆 Part 7: 方法大比拼

让我们系统地比较各种方法在高维数据下的表现。

In [ ]:
def compare_all_methods(X, T, Y, true_ate):
    """
    对比所有因果推断方法
    """
    results = []
    n = len(Y)
    
    # 1. 朴素估计
    naive_ate = Y[T==1].mean() - Y[T==0].mean()
    results.append({
        '方法': '朴素估计',
        'ATE': naive_ate,
        '偏差': naive_ate - true_ate,
        '说明': '直接对比均值'
    })
    
    # 2. OLS（带 T）
    from sklearn.linear_model import LinearRegression
    X_with_T = np.column_stack([T, X])
    ols = LinearRegression()
    ols.fit(X_with_T, Y)
    ols_ate = ols.coef_[0]
    results.append({
        '方法': 'OLS',
        'ATE': ols_ate,
        '偏差': ols_ate - true_ate,
        '说明': '高维下不稳定'
    })
    
    # 3. Lasso（有正则化偏差）
    lasso = LassoCV(cv=5, max_iter=10000)
    lasso.fit(X_with_T, Y)
    lasso_ate = lasso.coef_[0]
    results.append({
        '方法': 'Lasso',
        'ATE': lasso_ate,
        '偏差': lasso_ate - true_ate,
        '说明': '正则化偏差'
    })
    
    # 4. IPW
    lr = LogisticRegression(max_iter=1000, C=0.1)
    lr.fit(X, T)
    propensity = lr.predict_proba(X)[:, 1]
    propensity = np.clip(propensity, 0.01, 0.99)
    
    weights = T / propensity + (1 - T) / (1 - propensity)
    y1_ipw = (Y * weights * T).sum() / (weights * T).sum()
    y0_ipw = (Y * weights * (1 - T)).sum() / (weights * (1 - T)).sum()
    ipw_ate = y1_ipw - y0_ipw
    results.append({
        '方法': 'IPW',
        'ATE': ipw_ate,
        '偏差': ipw_ate - true_ate,
        '说明': '依赖倾向得分'
    })
    
    # 5. AIPW（无 Cross-fitting）
    g1 = Ridge(alpha=1.0)
    g0 = Ridge(alpha=1.0)
    g1.fit(X[T==1], Y[T==1])
    g0.fit(X[T==0], Y[T==0])
    mu1 = g1.predict(X)
    mu0 = g0.predict(X)
    
    aipw_scores = (mu1 - mu0) + T * (Y - mu1) / propensity - (1 - T) * (Y - mu0) / (1 - propensity)
    aipw_ate = aipw_scores.mean()
    results.append({
        '方法': 'AIPW (无 CF)',
        'ATE': aipw_ate,
        '偏差': aipw_ate - true_ate,
        '说明': '双重稳健但无交叉拟合'
    })
    
    # 6. DML（有 Cross-fitting）
    dml_result = double_ml_plr_complete(X, T, Y, n_folds=5)
    results.append({
        '方法': 'DML (有 CF)',
        'ATE': dml_result['tau'],
        '偏差': dml_result['tau'] - true_ate,
        '说明': '✅ 推荐'
    })
    
    return pd.DataFrame(results)

In [ ]:
# 运行对比
comparison_df = compare_all_methods(X, T, Y, TRUE_ATE)

print("🏆 方法大比拼")
print("=" * 60)
print(f"真实 ATE = {TRUE_ATE}")
print(f"数据维度: n={len(Y)}, p={X.shape[1]}")
print()
display(comparison_df)

# 可视化
fig, ax = plt.subplots(figsize=(10, 6))

colors = ['gray', 'gray', 'red', 'blue', 'orange', 'green']
bars = ax.barh(comparison_df['方法'], comparison_df['ATE'], color=colors, alpha=0.7)
ax.axvline(TRUE_ATE, color='black', linestyle='--', linewidth=2, label=f'真实 ATE = {TRUE_ATE}')
ax.set_xlabel('ATE 估计', fontsize=12)
ax.set_title('各方法 ATE 估计对比', fontsize=14)
ax.legend()

# 标注偏差
for bar, bias in zip(bars, comparison_df['偏差']):
    ax.text(bar.get_width() + 0.05, bar.get_y() + bar.get_height()/2,
           f'偏差: {bias:+.3f}', va='center', fontsize=10)

plt.tight_layout()
plt.show()

---

## 🎯 Part 8: 使用更强的 ML 模型

DML 的一个优势是可以使用任意 ML 模型！让我们试试随机森林。

In [ ]:
def double_ml_with_flexible_models(
    X: np.ndarray,
    T: np.ndarray,
    Y: np.ndarray,
    n_folds: int = 5,
    outcome_model_class = RandomForestRegressor,
    propensity_model_class = RandomForestClassifier,
    outcome_params: dict = None,
    propensity_params: dict = None
) -> Dict:
    """
    使用任意 ML 模型的 DML
    """
    n = len(Y)
    Y_residuals = np.zeros(n)
    T_residuals = np.zeros(n)
    
    outcome_params = outcome_params or {'n_estimators': 100, 'max_depth': 5, 'random_state': 42}
    propensity_params = propensity_params or {'n_estimators': 100, 'max_depth': 5, 'random_state': 42}
    
    kf = KFold(n_splits=n_folds, shuffle=True, random_state=42)
    
    for train_idx, test_idx in kf.split(X):
        X_train, X_test = X[train_idx], X[test_idx]
        T_train, T_test = T[train_idx], T[test_idx]
        Y_train, Y_test = Y[train_idx], Y[test_idx]
        
        # 结果模型
        g_model = outcome_model_class(**outcome_params)
        g_model.fit(X_train, Y_train)
        g_pred = g_model.predict(X_test)
        
        # 倾向得分模型
        m_model = propensity_model_class(**propensity_params)
        m_model.fit(X_train, T_train)
        m_pred = m_model.predict_proba(X_test)[:, 1]
        
        Y_residuals[test_idx] = Y_test - g_pred
        T_residuals[test_idx] = T_test - m_pred
    
    tau_hat = (Y_residuals * T_residuals).sum() / (T_residuals ** 2).sum()
    
    psi = (Y_residuals - tau_hat * T_residuals) * T_residuals
    J = (T_residuals ** 2).mean()
    var_tau = (psi ** 2).mean() / (n * J ** 2)
    se = np.sqrt(var_tau)
    
    return {
        'tau': tau_hat,
        'se': se,
        'ci_lower': tau_hat - 1.96 * se,
        'ci_upper': tau_hat + 1.96 * se
    }

In [ ]:
# 对比不同 ML 模型
print("🎯 DML + 不同 ML 模型")
print("=" * 50)

# Ridge (线性)
dml_ridge = double_ml_plr_complete(X, T, Y)
print(f"Ridge:          τ = {dml_ridge['tau']:.4f} ± {dml_ridge['se']:.4f}")

# Random Forest
dml_rf = double_ml_with_flexible_models(
    X, T, Y,
    outcome_model_class=RandomForestRegressor,
    propensity_model_class=RandomForestClassifier
)
print(f"Random Forest:  τ = {dml_rf['tau']:.4f} ± {dml_rf['se']:.4f}")

# Gradient Boosting
dml_gb = double_ml_with_flexible_models(
    X, T, Y,
    outcome_model_class=GradientBoostingRegressor,
    propensity_model_class=RandomForestClassifier,
    outcome_params={'n_estimators': 100, 'max_depth': 3, 'random_state': 42}
)
print(f"Gradient Boost: τ = {dml_gb['tau']:.4f} ± {dml_gb['se']:.4f}")

print(f"\n真实 ATE: {TRUE_ATE}")
print("\n💡 观察: 不同 ML 模型给出相似的结果，说明 DML 很稳健！")

---

## 💡 Part 9: 思考题

### 问题 1: DML 和普通 AIPW/DR 有什么区别？

**提示**: 想想 Cross-fitting 的作用

**你的答案:**

*（在这里写下你的思考...）*

---

### 问题 2: 为什么 DML 需要对 Y 和 T 都做残差化（Partialling Out）？

**提示**: 想想如果只对 Y 做残差化会怎样

**你的答案:**

*（在这里写下你的思考...）*

---

### 问题 3: Cross-fitting 的折数 K 如何选择？K=2 和 K=10 有什么区别？

**提示**: 想想偏差-方差权衡

**你的答案:**

*（在这里写下你的思考...）*

---

### 问题 4: DML 的关键假设是什么？什么时候 DML 会失效？

**提示**: 想想 unconfoundedness、overlap、model specification

**你的答案:**

*（在这里写下你的思考...）*

---

### 问题 5: 面试题：请用一分钟向面试官解释 DML 的核心思想

**你的答案:**

*（在这里写下你的回答...）*

---

### 📝 参考答案

<details>
<summary>点击展开</summary>

**问题 1**: DML 和 AIPW 的核心区别在于 **Cross-fitting**。AIPW 用同一份数据训练模型和计算效应，会有过拟合偏差；DML 通过 Cross-fitting 确保每个样本的预测都来自「未见过它」的模型，消除过拟合偏差。

**问题 2**: 如果只对 Y 做残差化（$Y - g(X)$），那 T 和 X 的相关性仍然存在，X 作为混淆变量仍会影响效应估计。只有同时对 Y 和 T 残差化，才能彻底「去除」X 的影响，这就是 Frisch-Waugh-Lovell 定理的延伸。

**问题 3**: 
- K=2: 每折只用 50% 数据训练，模型方差大，但偏差小
- K=10: 每折用 90% 数据训练，模型更稳定，但计算量大
- 通常 K=5 是一个好的平衡点

**问题 4**: DML 的关键假设：
1. **Unconfoundedness**: 无未观测混淆
2. **Overlap**: 倾向得分不能太极端
3. **模型收敛速度**: g(X) 和 m(X) 的估计误差衰减足够快（$o(n^{-1/4})$）

**问题 5**: 面试回答模板：

「DML 是一种用机器学习估计因果效应的方法。核心思想是：
1. 先用 ML 模型把协变量 X 对 Y 和 T 的影响去掉，得到残差
2. 用残差做回归，估计 T 对 Y 的因果效应
3. 通过 Cross-fitting 避免过拟合偏差

这样做的好处是：可以用任意 ML 模型处理高维数据，同时保证因果效应估计的无偏性和有效推断。」

</details>

---

## 🎤 高频面试题详解

### 面试题 1: DML 和普通 DR/AIPW 有什么区别？为什么需要 Cross-fitting？

**标准回答**:

**核心区别**: Cross-fitting

**问题来源**: 
当我们用机器学习模型（尤其是灵活的非参数模型）时，如果用**同一份数据**训练模型和预测：

$$\text{残差} = Y_i - \hat{g}(X_i)$$

其中 $\hat{g}$ 在训练时"见过" $Y_i$，会导致：
- 残差被**低估**（overfitting bias）
- 标准误失效
- 置信区间过窄

**Cross-fitting 的解决方案**:
1. 把数据分成 K 折
2. 对每一折，用其他 K-1 折训练模型
3. 用训练好的模型预测当前折

这样每个 $Y_i$ 的预测都来自「没见过它」的模型！

**关键洞察**: 这类似 cross-validation，但目的不是选模型，而是**去除过拟合偏差**。

---

### 面试题 2: 什么是 Neyman 正交性？为什么它重要？

**标准回答**:

**定义**: Neyman 正交性是指矩函数对 nuisance 参数的导数为 0：

$$\frac{\partial}{\partial \eta} E[\psi(W; \tau_0, \eta)] \Big|_{\eta=\eta_0} = 0$$

**重要性**: 
- 在经典方法中，nuisance 参数估计误差对目标参数估计的影响是**一阶的** $O(||\hat{\eta} - \eta||)$
- 在 Neyman 正交的方法中，影响是**二阶的** $O(||\hat{\eta} - \eta||^2)$

**直观比喻**:
- 普通方法：小石子砸到车（一阶影响）
- Neyman 正交：小石子被弹开（二阶影响，几乎没影响）

**实际意义**:
- 允许使用正则化 ML 模型（Lasso, Ridge, RF）
- 即使模型有偏差，对因果效应估计影响很小
- 可以达到 $\sqrt{n}$-一致性和渐近正态性

---

### 面试题 3: DML 的关键假设是什么？什么时候会失效？

**标准回答**:

**关键假设**:

1. **Unconfoundedness (无混淆)**:
   $$(Y(0), Y(1)) \perp T | X$$
   - 如果有重要的未观测混淆，DML 无法解决
   
2. **Overlap/Positivity (重叠)**:
   $$0 < P(T=1|X) < 1$$
   - 如果某些 X 值下 T 几乎确定，无法估计效应
   
3. **模型收敛速度**:
   $$||\hat{g} - g|| = o_p(n^{-1/4}), \quad ||\hat{m} - m|| = o_p(n^{-1/4})$$
   - Nuisance 模型不能太差

**失效场景**:

| 场景 | 后果 | 解决方案 |
|------|------|----------|
| 有未观测混淆变量 | 估计有偏 | 工具变量、DID、RDD |
| 倾向得分接近 0 或 1 | 估计不稳定 | 修剪样本、限制推断范围 |
| 样本量太小（n < 500） | 方差大、推断不准 | 用参数方法或收集更多数据 |
| 模型收敛太慢 | 渐近正态性不成立 | 用更灵活的模型或特征工程 |

---

### 面试题 4: Cross-fitting 的折数 K 如何选择？

**标准回答**:

**权衡**:

| K值 | 训练集大小 | 模型质量 | 计算成本 | 推荐场景 |
|-----|-----------|---------|---------|----------|
| K=2 | 50% | 方差大 | 低 | 快速原型 |
| K=5 | 80% | ✅ 平衡 | 中 | **推荐** |
| K=10 | 90% | 方差小 | 高 | 大数据集 |
| K=n (LOOCV) | ~100% | 最小方差 | 很高 | 小数据（n<500） |

**理论考虑**:
- K 太小：每折训练集小，$\hat{g}$, $\hat{m}$ 估计不准
- K 太大：计算量大，交叉验证方差增加

**实践建议**:
- **默认 K=5**: 在理论性质和计算效率间的最佳平衡
- n < 1000 时可以用 K=10
- n > 10000 时 K=2 或 K=3 就够了

**面试加分点**: 提到 repeated cross-fitting（重复多次 cross-fitting 取平均）可以进一步降低方差。

---

### 面试题 5: 为什么 DML 需要对 Y 和 T 都做残差化？

**标准回答**:

这涉及 **Frisch-Waugh-Lovell (FWL) 定理**！

**FWL 定理**: 在多元回归中
$$Y = \tau T + X'\beta + \epsilon$$

$\tau$ 的 OLS 估计等价于：
1. 对 Y 和 T 分别对 X 回归，得到残差 $\tilde{Y}$, $\tilde{T}$
2. 用 $\tilde{Y}$ 对 $\tilde{T}$ 做简单回归

**为什么两个都要残差化？**

- **只残差化 Y**: $\tilde{Y} = Y - g(X)$
  - 但 T 和 X 仍相关，X 仍是混淆变量！
  - 相当于只做了一半的控制

- **只残差化 T**: $\tilde{T} = T - m(X)$  
  - Y 中仍包含 X 的影响
  - 无法分离 T 的纯效应

- **两个都残差化**:
  - $\tilde{Y}$ 去除了 X 对结果的直接影响
  - $\tilde{T}$ 去除了 X 对处理的影响  
  - 残差间的关系就是 T 对 Y 的因果效应！

**直观比喻**: 
要测量「咖啡」对「工作效率」的影响：
- 只控制 Y：去除睡眠对效率的影响，但没控制睡眠对喝咖啡的影响
- 只控制 T：去除睡眠对喝咖啡的影响，但没控制睡眠对效率的直接影响
- 两个都控制：完全分离出咖啡的纯效应！

---

---

## 📋 总结

### 核心公式

**DML for Partially Linear Model:**

$$\hat{\tau}_{DML} = \frac{\sum_{i=1}^{n}(Y_i - \hat{g}_{-k(i)}(X_i))(T_i - \hat{m}_{-k(i)}(X_i))}{\sum_{i=1}^{n}(T_i - \hat{m}_{-k(i)}(X_i))^2}$$

其中 $\hat{g}_{-k(i)}$ 表示在不包含第 $i$ 个样本的折上训练的模型。

### 关键概念

| 概念 | 含义 | 为什么重要 |
|------|------|------------|
| 正则化偏差 | ML 模型的正则化会压缩因果效应 | 高维数据下的核心问题 |
| Neyman 正交 | 构造对 nuisance 参数不敏感的估计 | DML 的理论基础 |
| Cross-fitting | 用不同折的数据训练和预测 | 消除过拟合偏差 |
| Double Residualization | 对 Y 和 T 都做残差化 | 去除 X 的影响 |

### DML vs 其他方法

| 方法 | 优点 | 缺点 | 适用场景 |
|------|------|------|----------|
| OLS | 简单 | 高维下不稳定 | 低维 |
| Lasso | 高维可用 | 正则化偏差 | 纯预测 |
| IPW | 无需结果模型 | 权重极端 | 中等维度 |
| AIPW | 双重稳健 | 过拟合偏差 | 中等维度 |
| **DML** | 高维 + 无偏 + 有效推断 | 计算量大 | **高维因果推断** |

### 面试要点

1. DML 解决**高维数据**下的因果推断问题
2. 核心是 **Cross-fitting** + **双重残差化**
3. 可以使用**任意 ML 模型**作为 nuisance 估计器
4. 保证渐近正态性，可以做**有效统计推断**

---

**「DML 就像做饭前先把食材洗干净（去除 X 的影响），再开始烹饪（估计因果效应）。」**

🎉 恭喜完成 Double ML 深度剖析！